In [1]:
import numpy as np
import pandas as pd
#读取Excel数据，统一先按照字符串读入，之后转换
file='./朝阳医院2018年销售数据.xlsx'
xls = pd.ExcelFile(file, dtype='object')
salesDf = xls.parse('Sheet1',dtype='object')
#打印出前5行，以确保数据运行正常
salesDf.head()


,购药时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01 星期五,001616528,236701,强力VC银翘片,6,82.8,69
1,2018-01-02 星期六,001616528,236701,清热解毒口服液,1,28,24.64
2,2018-01-06 星期三,0012602828,236701,感康,2,16.8,15
3,2018-01-11 星期一,0010070343428,236701,三九感冒灵,1,28,28
4,2018-01-15 星期五,00101554328,236701,三九感冒灵,8,224,208


In [2]:
salesDf.shape
salesDf.dtypes

购药时间    object
社保卡号    object
商品编码    object
商品名称    object
销售数量    object
应收金额    object
实收金额    object
dtype: object

In [3]:
colNameDict = {'购药时间':'销售时间'}
salesDf.rename(columns = colNameDict,inplace=True)
salesDf.head()

,销售时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01 星期五,001616528,236701,强力VC银翘片,6,82.8,69
1,2018-01-02 星期六,001616528,236701,清热解毒口服液,1,28,24.64
2,2018-01-06 星期三,0012602828,236701,感康,2,16.8,15
3,2018-01-11 星期一,0010070343428,236701,三九感冒灵,1,28,28
4,2018-01-15 星期五,00101554328,236701,三九感冒灵,8,224,208


In [4]:
salesDf=salesDf.dropna(subset=['销售时间','社保卡号'],how='any')
print('删除缺失后大小',salesDf.shape)

删除缺失后大小 (6575, 7)


In [5]:
#字符串转换为数值（浮点型）
salesDf['销售数量'] = salesDf['销售数量'].astype('float')
salesDf['应收金额'] = salesDf['应收金额'].astype('float')
salesDf['实收金额'] = salesDf['实收金额'].astype('float')

In [7]:
def splitSaletime(timeColSer):
    timeList=[]
    for value in timeColSer:
        #例如2018-01-01 星期五，分割后为：2018-01-01
        dateStr=value.split(' ')[0]
        timeList.append(dateStr)
    
    #将列表转行为一维数据Series类型
    timeSer=pd.Series(timeList)
    return timeSer

In [8]:
timeSer=salesDf.loc[:,'销售时间']
dateSer=splitSaletime(timeSer)
salesDf.loc[:,'销售时间']=dateSer

'''
数据类型转换:字符串转换为日期
'''
#errors='coerce' 如果原始数据不符合日期的格式，转换后的值为空值NaT
#format 是你原始数据中日期的格式
salesDf.loc[:,'销售时间']=pd.to_datetime(salesDf.loc[:,'销售时间'],
                                    format='%Y-%m-%d', 
                                    errors='coerce')

salesDf.dtypes

销售时间    datetime64[ns]
社保卡号            object
商品编码            object
商品名称            object
销售数量           float64
应收金额           float64
实收金额           float64
dtype: object

In [9]:
'''
转换日期过程中不符合日期格式的数值会被转换为空值，
这里删除列（销售时间，社保卡号）中为空的行
'''
salesDf=salesDf.dropna(subset=['销售时间','社保卡号'],how='any')


In [10]:
'''
by：按那几列排序
ascending=True 表示降序排列，
ascending=False表示升序排列
'''
#按销售日期进行升序排列
salesDf=salesDf.sort_values(by='销售时间',ascending=True)

#重命名行名（index）：排序后的列索引值是之前的行号，需要修改成从0到N按顺序的索引值
salesDf=salesDf.reset_index(drop=True)
salesDf.head()

,销售时间,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额
0,2018-01-01,001616528,236701,强力VC银翘片,6.0,82.8,69.0
1,2018-01-01,0010616728,865099,硝苯地平片(心痛定),2.0,3.4,3.0
2,2018-01-01,0010073966328,861409,非洛地平缓释片(波依定),5.0,162.5,145.0
3,2018-01-01,0010073966328,866634,硝苯地平控释片(欣然),6.0,111.0,92.5
4,2018-01-01,0010014289328,866851,缬沙坦分散片(易达乐),1.0,26.0,23.0


In [11]:
salesDf.describe()

,销售数量,应收金额,实收金额
count,6549.000000,6549.000000,6549.000000
mean,2.384486,50.449076,46.284370
std,2.375227,87.696401,81.058426
min,-10.000000,-374.000000,-374.000000
25%,1.000000,14.000000,12.320000
50%,2.000000,28.000000,26.500000
75%,2.000000,59.600000,53.000000
max,50.000000,2950.000000,2650.000000


In [16]:
#删除异常值：通过条件判断筛选出数据
#查询条件
querySer=salesDf.loc[:,'销售数量']>0
#应用查询条件
print('删除异常值前：',salesDf.shape)
salesDf=salesDf.loc[querySer,:]
print('删除异常值后：',salesDf.shape)

删除异常值前： (6506, 7)
删除异常值后： (6506, 7)


,销售数量,应收金额,实收金额
count,6506.000000,6506.000000,6506.000000
mean,2.405626,50.927897,46.727653
std,2.364565,87.650282,80.997726
min,1.000000,1.200000,0.030000
25%,1.000000,14.000000,12.600000
50%,2.000000,28.000000,27.000000
75%,2.000000,59.600000,53.000000
max,50.000000,2950.000000,2650.000000


In [24]:
'''
总消费次数：同一天内，同一个人发生的所有消费算作一次消费
#根据列名（销售时间，社区卡号），如果这两个列值同时相同，只保留1条，将重复的数据删除
'''

kpi1_Df=salesDf.drop_duplicates(subset=['销售时间', '社保卡号'])

#总消费次数：有多少行
totalI=kpi1_Df.shape[0]

print('总消费次数=',totalI)


#月份数,运算符"//"表示取整除，返回商的整数部分
startTime=kpi1_Df.loc[0,'销售时间']
endTime=kpi1_Df.loc[totalI-1,'销售时间']
daysI=(endTime-startTime).days
monthsI=daysI//30
remain = daysI - 30 * monthsI
if(remain > 0):
    monthsI = monthsI + 1
print('月份数：',monthsI)


#业务指标1：月均消费次数=总消费次数 / 月份数
kpi1_I=totalI // monthsI
print('业务指标1：月均消费次数=',kpi1_I)

总消费次数= 5342
163
月份数： 6
业务指标1：月均消费次数= 890


In [25]:
totalMoneyF=salesDf.loc[:,'实收金额'].sum()
#月均消费金额
monthMoneyF=totalMoneyF / monthsI
print('业务指标2：月均消费金额=',monthMoneyF)

业务指标2：月均消费金额= 50668.351666666305


In [26]:
'''
totalMoneyF：总消费金额
totalI：总消费次数
'''
pct=totalMoneyF / totalI
print('客单价：',pct)

客单价： 56.9094178210404
